# Playground for messing around with pandas

In [1]:
# import dependencies
import pandas as pd


In [2]:
# create a dataset of UFO sightings as a pandas dataframe object
# We can specify the columns we want using usecols argument
# We use the dtype argument to explicitly state the dtype of some columns in a dictionary
data = pd.read_csv('data/ufo_sighting_data.csv',
                   usecols=['Date_time', 'city', 'state/province', 'country', 'UFO_shape',
                            'length_of_encounter_seconds', 'described_duration_of_encounter',
                            'date_documented', 'latitude', 'longitude'],
                   dtype={ 'length_of_encounter_seconds': object, 'latitude': object })


# We will cast this column to a float as the dataset itself contains some artifacts
data['length_of_encounter_seconds'] = pd.to_numeric(data['length_of_encounter_seconds'], errors='coerce')

## A quick look at our data

In [3]:
# The shape of all our data
# a tuple where the 1st int is the number of records or observations in our dataset
# the second number is the number of variables or features each observation has

data.shape

(80332, 10)

In [4]:
# column names
data.columns

Index(['Date_time', 'city', 'state/province', 'country', 'UFO_shape',
       'length_of_encounter_seconds', 'described_duration_of_encounter',
       'date_documented', 'latitude', 'longitude'],
      dtype='object')

In [5]:
# Lets look at the data types of our columns
# Object columns basically just means string

print(data.dtypes)

Date_time                           object
city                                object
state/province                      object
country                             object
UFO_shape                           object
length_of_encounter_seconds        float64
described_duration_of_encounter     object
date_documented                     object
latitude                            object
longitude                          float64
dtype: object


In [6]:
# columns can be renamed like this
data.rename(columns={ 'UFO_shape': 'shape' }, inplace=True)

# We can also rename all colums quickly like so
data.columns = ['Date_time', 'city', 'state/province', 'country', 'UFO_shape',
       'duration', 'described_duration_of_encounter',
       'date_documented', 'latitude', 'longitude']

In [7]:
# The head method gives us a quick look at our data by returning the first 5 records

print(data.head())

          Date_time                  city state/province country UFO_shape  \
0  10/10/1949 20:30            san marcos             tx      us  cylinder   
1  10/10/1949 21:00          lackland afb             tx     NaN     light   
2  10/10/1955 17:00  chester (uk/england)            NaN      gb    circle   
3  10/10/1956 21:00                  edna             tx      us    circle   
4  10/10/1960 20:00               kaneohe             hi      us     light   

   duration described_duration_of_encounter date_documented    latitude  \
0    2700.0                      45 minutes       4/27/2004  29.8830556   
1    7200.0                         1-2 hrs      12/16/2005    29.38421   
2      20.0                      20 seconds       1/21/2008        53.2   
3      20.0                        1/2 hour       1/17/2004  28.9783333   
4     900.0                      15 minutes       1/22/2004  21.4180556   

    longitude  
0  -97.941111  
1  -98.581082  
2   -2.916667  
3  -96.645833  


In [8]:
# A helpful way to get an overview of the actual data
# Works with numeric data, not so useful with longitudinal data

print(data.describe())
print(data.describe(include=['object']))

           duration     longitude
count  8.032900e+04  80332.000000
mean   9.017226e+03    -86.772885
std    6.202284e+05     39.697205
min    1.000000e-03   -176.658056
25%    3.000000e+01   -112.073333
50%    1.800000e+02    -87.903611
75%    6.000000e+02    -78.755000
max    9.783600e+07    178.441900
             Date_time     city state/province country UFO_shape  \
count            80332    80332          74535   70662     78400   
unique           69586    19900             67       5        29   
top     7/4/2010 22:00  seattle             ca      us     light   
freq                36      525           9655   65114     16565   

       described_duration_of_encounter date_documented    latitude  
count                            80332           80332       80332  
unique                            8304             317       18421  
top                          5 minutes      12/12/2009  47.6063889  
freq                              4716            1510         581  


## Series manipulation

### Selecting

In [9]:
# Simply use bracket notation to select a series from a dataframe
states = data['state/province']

# we can also use dot notation
durations = data.duration
print('States: ')
print(states.head(), end='\n\n')

print('Durations: ')
print(durations.head())
print(type(durations))

States: 
0     tx
1     tx
2    NaN
3     tx
4     hi
Name: state/province, dtype: object

Durations: 
0    2700.0
1    7200.0
2      20.0
3      20.0
4     900.0
Name: duration, dtype: float64
<class 'pandas.core.series.Series'>


### Note
Dot notation will not work when callign attributes with the same name as reserved dataframe properties and methods. e.g 'shape', 'head'

### Creating

In [10]:
# Lets say we want to combine the city and state like so: city, state
# we must use bracket notation
data['location'] = data.city + ', ' + data['state/province']
print(data.location.head())

0      san marcos, tx
1    lackland afb, tx
2                 NaN
3            edna, tx
4         kaneohe, hi
Name: location, dtype: object


### Dropping

In [11]:
# You can drop a column by passing in a string or many by passing a list of strings
# Axis specifies that we want to drop a column
data.drop('described_duration_of_encounter', axis=1, inplace=True)

In [12]:
# We can drop rows by setting axis to 0 and specifying the record ID or row number
print('before: ' + str(data.shape))

data.drop([0, 1, 2, 3, 4], axis=0, inplace=True)

print('Dropping 5 rows...')
print('after: ' + str(data.shape))

before: (80332, 10)
Dropping 5 rows...
after: (80327, 10)


### String manipulation

In [13]:
# We can manipulate all the strings inside a series using these built in string methods
# Lets say we want to upper() every state/province 
# we just call str() on the series and then a string method

data['state/province'].str.upper()

# string methods can also be chained but since a series is returned we must use str() again
# Here we get rid of brackets and their contents as well as capitalize the first letter of every word.
# replace() can take a string or a regex

data['city'].str.replace('\([a-z/-]+\)', '').str.title()[0:20]


5               Bristol
6              Penarth 
7               Norwalk
8             Pell City
9              Live Oak
10            Hawthorne
11              Brevard
12             Bellmore
13           Manchester
14            Lexington
15        Harlan County
16      West Bloomfield
17              Niantic
18          Bermuda Nas
19               Hudson
20             Cardiff 
21               Hudson
22     North Charleston
23            Washougal
24    Stoke Mandeville 
Name: city, dtype: object

### Sorting 

In [14]:
# We can sort based on a series 
print(data.city.sort_values().head())

# change order by passing in ascending=False
print()
print(data.city.sort_values(ascending=False).head())

9907                             &ccedil;anakkale (turkey)
1076                  &iacute;safj&ouml;r&eth;ur (iceland)
24320                                &ouml;lmstad (sweden)
39987    1-25 corridor (southbound&#44 65 miles north n...
7463                                     100 mile (canada)
Name: city, dtype: object

18796                   zwolle
64220       zwevegem (belgium)
37622    zutphen (netherlands)
79296                 zumbrota
19979                 zumbrota
Name: city, dtype: object


### Sorting The dataframe

In [15]:
# We can sort the dataframe by the series
print(data.sort_values('city', ascending=False).head())

              Date_time                   city state/province country  \
18796  12/18/2002 17:30                 zwolle             la      us   
64220   8/10/2001 20:45     zwevegem (belgium)            NaN     NaN   
37622   4/18/2010 23:00  zutphen (netherlands)            NaN     NaN   
79296    9/5/1999 20:50               zumbrota             mn      us   
19979  12/23/2000 16:30               zumbrota             mn      us   

      UFO_shape  duration date_documented    latitude  longitude      location  
18796     other    1260.0       12/2/2013  31.6313889 -93.643889    zwolle, la  
64220   unknown    1500.0       8/12/2001   50.812926   3.332698           NaN  
37622    sphere     120.0       4/18/2012   52.142736   6.196058           NaN  
79296    circle       4.0       10/2/1999  44.2941667 -92.668889  zumbrota, mn  
19979     cigar      30.0       1/17/2004  44.2941667 -92.668889  zumbrota, mn  


In [16]:
# We can sort dataframes by many series by passing in a list to sort_values()
print(data.sort_values(['Date_time', 'city'], ascending=False).head())

            Date_time       city state/province country UFO_shape  duration  \
80331  9/9/2013 23:00     edmond             ok      us     cigar    1020.0   
80330  9/9/2013 22:20     vienna             va      us    circle       5.0   
80329  9/9/2013 22:00       napa             ca      us     other    1200.0   
80328  9/9/2013 22:00      boise             id      us    circle    1200.0   
80327  9/9/2013 21:15  nashville             tn      us     light     600.0   

      date_documented    latitude   longitude       location  
80331       9/30/2013  35.6527778  -97.477778     edmond, ok  
80330       9/30/2013  38.9011111  -77.265556     vienna, va  
80329       9/30/2013  38.2972222 -122.284444       napa, ca  
80328       9/30/2013  43.6136111 -116.202500      boise, id  
80327       9/30/2013  36.1658333  -86.784444  nashville, tn  


## Filtering Dataframes

In [17]:
# We can filter our results with a simple one line command as follows
ca_sightings = data[data['state/province'] == 'ca']

# explanation for this syntax: 
# pandas will create a logical series using the coniditional we provided
# The logical series will then be used to filter out the records that were false in the series
# Think of using a for loop and that conditional being inside of it

# preview of the logical series
print((data['state/province'] == 'ca').head())
print(ca_sightings.shape)
ca_sightings.head()

5    False
6    False
7    False
8    False
9    False
Name: state/province, dtype: bool
(9655, 10)


,Date_time,city,state/province,country,UFO_shape,duration,date_documented,latitude,longitude,location
10,10/10/1968 13:00,hawthorne,ca,us,circle,300.0,10/31/2003,33.9163889,-118.351667,"hawthorne, ca"
30,10/10/1979 22:00,san diego,ca,us,oval,180.0,8/5/2001,32.7152778,-117.156389,"san diego, ca"
45,10/10/1989 00:00,calabasas,ca,us,disk,300.0,12/14/2004,34.1577778,-118.637500,"calabasas, ca"
63,10/10/1995 22:40,oakland,ca,us,NaN,60.0,11/2/1999,37.8044444,-122.269722,"oakland, ca"
72,10/10/1998 02:30,hollywood,ca,us,changing,300.0,11/1/1998,34.0983333,-118.325833,"hollywood, ca"


In [18]:
# if we want a specific column / columns from our query we can use dot notation or bracket notation to specify them 
# However this can cause funny behaviour, so best practice is to use loc[]

ca_sighting_durations = data.loc[data['state/province'] == 'ca', 'duration']
print(ca_sighting_durations[0:10])

# hell yeah!

10      300.0
30      180.0
45      300.0
63       60.0
72      300.0
82     3600.0
85     3600.0
87       90.0
105     120.0
107     900.0
Name: duration, dtype: float64


In [19]:
# Lets have a bit of fun
ca_sighting_durations.describe()

count    9.654000e+03
mean     3.928781e+03
std      1.356938e+05
min      1.000000e-01
25%      3.000000e+01
50%      1.800000e+02
75%      6.000000e+02
max      1.052640e+07
Name: duration, dtype: float64

### One hot encoded series

In [20]:
# We can one hot encode boolean series simply by casting it to an integer type like so
# This is useful for training machine learning models that require numeric input

(data['duration'] > 800).astype(int).head()

5    0
6    0
7    1
8    0
9    0
Name: duration, dtype: int64

### Multiple filters

In [21]:
# To apply multiple filters:
# Wrap each logical series in paranthesis
# Use & | for operators between them

ca_circular_sightings = data[(data['state/province'] == 'ca') & (data['UFO_shape'] == 'circle')]
print(ca_circular_sightings.shape)
ca_circular_sightings.head()

(899, 10)


,Date_time,city,state/province,country,UFO_shape,duration,date_documented,latitude,longitude,location
10,10/10/1968 13:00,hawthorne,ca,us,circle,300.0,10/31/2003,33.9163889,-118.351667,"hawthorne, ca"
87,10/10/1999 20:35,hayward,ca,us,circle,90.0,11/20/2001,37.6688889,-122.079722,"hayward, ca"
105,10/10/2001 20:35,hayward,ca,us,circle,120.0,11/20/2001,37.6688889,-122.079722,"hayward, ca"
354,10/11/2004 14:00,arroyo grande,ca,us,circle,60.0,12/14/2004,35.1186111,-120.589722,"arroyo grande, ca"
385,10/11/2006 21:32,la crescenta,ca,us,circle,3.0,10/30/2006,34.2241667,-118.239167,"la crescenta, ca"


### Multiple OR choices

In [22]:
# If we needed to filter a dataframe by series where we had a handful of options then we can use a series method
# called isin() and pass it a list of the values we want to keep

my_cities_sightings = data[data.city.isin(['san diego', 'oakland', 'edna'])]
print(my_cities_sightings.shape)
my_cities_sightings.sort_values('city')[0:4]


(404, 10)


,Date_time,city,state/province,country,UFO_shape,duration,date_documented,latitude,longitude,location
12866,11/25/1985 00:00,edna,tx,us,unknown,240.0,1/21/2008,28.9783333,-96.645833,"edna, tx"
43242,5/19/2012 21:00,edna,tx,us,other,3600.0,5/29/2012,28.9783333,-96.645833,"edna, tx"
37290,4/16/2012 18:00,oakland,ca,us,disk,300.0,5/13/2012,37.8044444,-122.269722,"oakland, ca"
64032,7/9/2010 13:00,oakland,ca,us,cylinder,900.0,7/28/2010,37.8044444,-122.269722,"oakland, ca"


In [23]:
my_cities_sightings.sort_values('city')[-4:]

,Date_time,city,state/province,country,UFO_shape,duration,date_documented,latitude,longitude,location
22852,12/6/1999 16:45,san diego,ca,us,unknown,300.0,12/16/1999,32.7152778,-117.156389,"san diego, ca"
22521,12/5/1999 23:00,san diego,ca,us,fireball,7.0,12/16/1999,32.7152778,-117.156389,"san diego, ca"
24750,1/5/2007 21:15,san diego,ca,us,light,1.0,2/1/2007,32.7152778,-117.156389,"san diego, ca"
80316,9/9/2013 09:51,san diego,ca,us,light,4.0,9/30/2013,32.7152778,-117.156389,"san diego, ca"


## Groupby

We should use grouby methods when our questions consist of something like "For each group i want the average of...". There are many groupby methods which pandas provides such as 

* mean
* sum
* max
* min

In [29]:
mean_sighting_time_by_country = data.groupby('country').duration.mean()
print(mean_sighting_time_by_country)
print('Fascinating')

country
au      538
ca     3000
de      105
gb     1904
us    65109
Name: duration, dtype: int64
Fascinating


We can specify multiple aggregator methods by running the agg method on the feature

In [ ]:
co